In [8]:
import os
import pandas as pd
import numpy as np
import scipy.io
from scipy.optimize import minimize
from itertools import combinations
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

class BLP(object):
    '''
    Jia Yan
    02/21/2023
    '''
    
    def __init__(self, path_data, file_data, path_str_data, file_str_data, ndraws=500, tol_fp=1e-12):
        '''
        set parameters
        '''
        self.ndraws = ndraws # number of random draws in monte-carlo integration
        self.tol_fp = tol_fp # convergence tolerance level in nested fixed-point iteration
        
        '''
        construct data 
        '''
        self.df = self._initialize_data(path_data, file_data, path_str_data, file_str_data)
        self.nmarkets = int(self.df['trend'].max()) + 1 # a market is a year
        self.y_fixed = np.log(self.df['share']/self.df['outshr']) # Dependent variable in demand models without random coefficients
                            
        '''
        demand side specification
        '''
        attributes = ['const', 'hpwt', 'air', 'mpd', 'space', 'price'] # demand side variables with same order as in the BLP paper
        self.attributes_random = ['const', 'hpwt', 'air', 'mpd', 'space'] # variables with random coefficients
        self.Xmat = self.df[attributes].to_numpy() # X-matrix in demand model
        
        '''
        construct demand-side instruments
        '''
        self.Zmat_D =  self._initialize_IV_D(self.df) # matrix of demand side instruments
        self.weight_mat_D = np.linalg.inv(np.matmul(np.transpose(self.Zmat_D), self.Zmat_D)) # weighting matrix in GMM estimation
        self.project_mat = self._initialize_2sls_D() # inv(X'PX)(X'P) in which P = Z*inv(Z'Z)*Z'  
        
        '''
        supply side specification
        '''
        mc = ['const', 'mpg', 'air', 'space', 'hpwt', 'trend'] # marginal cost variables
        self.Mcmat = self._initialize_mc(mc) # matrix of variables in marginal cost equation
        wmat_T = np.transpose(self.Mcmat)
        mom = np.matmul(wmat_T, self.Mcmat)
        self.PWMAT = np.matmul(np.linalg.inv(mom), wmat_T)

        '''
        construct supply-side instruments
        '''
        self.Zmat_S = self._initialize_IV_S() 
        self.weight_mat_S = np.linalg.inv(np.matmul(np.transpose(self.Zmat_S), self.Zmat_S)) # weighting matrix in GMM estimation
        
        ''''
        Take standard normal draws for approximating integrals in market share and mark-up computations.
        Better to use halton draws
        '''
        self.draws = np.random.randn(self.nmarkets, self.ndraws, len(self.attributes_random))    
    
    def _initialize_data(self, path_data, file_data, path_str_data, file_str_data):
        data = scipy.io.loadmat(os.path.join(path_data, file_data))
        model_name = scipy.io.loadmat(os.path.join(path_str_data, file_str_data))['model_name']
        v_list = ['outshr', 'const', 'mpd', 'mpg', 'air', 'space', 'hpwt', 'price', 'trend']
        df = data['share']
        for item in v_list:
            df = np.concatenate([df, data[item]], axis=1)
        df = pd.DataFrame(df, columns = ['share'] + v_list)
        df['model_name'] = model_name
        df['maker'] = df['model_name'].transform(lambda x: x[0:2])
        df = df.sort_values(by=['trend', 'maker'], ascending=[True, True]) # group products by market and by firm
        return df
    
    def _initialize_IV_D(self, df):
        # demand side instruments for price: sum of attributes of other products from the same firm and of products from rival firms
        z_list = ['const'] # creat instruments of price from this list
        IV_list = ['const', 'mpd', 'air', 'space', 'hpwt'] # the first part of IV are exogenous regressors
        for var in z_list:
            name_own = var + "_" + "z" + "_" + "own"
            IV_list.append(name_own)
            name_rival = var + "_" + "z" + "_" + "rival"
            IV_list.append(name_rival)
            
            df[name_own] = df.groupby(['trend', 'maker'])[var].transform(lambda x: x.sum())
            df[name_own] = df[name_own] - self.df[var]
            
            df['junk']= df.groupby(['trend'])[var].transform(lambda x: x.sum()) - self.df[var]
            df[name_rival] = df['junk'] - df[name_own]
        
        return df[IV_list].to_numpy() ## the matrix of demand-side instruments
    
    def _initialize_2sls_D(self):
        pz = np.matmul(self.Zmat_D, self.weight_mat_D)
        pz = np.matmul(pz, np.transpose(self.Zmat_D))
        project_mat = np.matmul(np.transpose(self.Xmat), pz)
        project_mat = np.matmul(project_mat, self.Xmat)
        project_mat = np.linalg.inv(project_mat)
        project_mat = np.matmul(project_mat, np.transpose(self.Xmat))
        return np.matmul(project_mat, pz)
    
    def _initialize_mc(self, vlist):
        df = self.df[vlist].copy()
        df['mpg'] = np.log(df['mpg'])
        df['space'] = np.log(df['space'])
        df['hpwt'] = np.log(df['hpwt'])
        return df.to_numpy()
    
    def _initialize_ols_S(self): 
        mom = np.matmul(np.transpose(self.Mcmat), self.Mcmat)
        return np.matmul(np.linalg.inv(mom), self.Mcmat)
    
    def _initialize_IV_S(self):
        def util(pair):
            return np.multiply(pair[0], pair[1])
        m = self.Mcmat[:, 1:] # remove constant term
        l = [m[:,i] for i in range(m.shape[1])]
        pairs = combinations(l, 2)
        zmat = self.Mcmat # the first part of IVs are exo. regressors
        interactions = np.array(list(map(util, pairs)))
        interactions = np.transpose(interactions)
        return np.hstack((zmat, interactions))
        
    def ols(self):
        '''
        replicate the first column of table 3
        '''
        y = np.log(self.df['share']/self.df['outshr'])
        return sm.OLS(self.y_fixed, self.Xmat).fit()
        
    def iv(self):
        '''
        replicate the second column of table 3
        '''
        #return np.matmul(self.project_mat, self.y_fixed)
        return IV2SLS(self.y_fixed, self.Xmat, self.Zmat_D).fit()
    
    def hedonic_price(self):
        '''replicate the third column of table 3'''
        y = np.log(self.df['price'])
        return sm.OLS(y, self.Mcmat).fit()
    
    def markup_conditional(self, udic):
        v = np.exp(udic['delta'] + (udic['rdraw'] * udic['xv']).sum(axis=1))
        s = v / (1 + np.sum(v))
        cut = 0
        dmat = []
        for i in udic['jf']:
            sg = s[cut:cut+i]
            dmat.append(np.diag(sg) - np.outer(sg,sg))
            cut = cut + i
        return {'dmat': dmat, 'shares': s}
    
    def share_conditional(self, udic):
        v = np.exp(udic['delta'] + (udic['rdraw'] * udic['xv']).sum(axis=1))
        return v / (1 + np.sum(v))
        
    def market_share(self, mid, delta, xv, jf, markup=False):
        draws = self.draws[mid]
        inputs = [{'delta': delta, 'xv':xv, 'rdraw':draws[r], 'jf': jf} for r in range(self.ndraws)]
        if markup is False:
            out = map(self.share_conditional, inputs)
            return (1/self.ndraws) * np.sum(list(out), axis=0) 
        else:
            out = list(map(self.markup_conditional, inputs))
            d = [i['dmat'] for i in out]
            s = [i['shares'] for i in out] 
            dmat = [(1/self.ndraws) * item for item in map(sum, zip(*d))]
            shares = (1/self.ndraws) * np.sum(s, axis=0)
            return {'dmat': dmat, 'shares':shares}
       
    def fixed_point(self, pack):
        mid = pack['mid']
        df = pack['df']
        sigmas = pack['sigmas']
        s0 = df['share'].to_numpy()
        xv = sigmas * df[self.attributes_random].to_numpy()
        jf = [len(g) for _, g in df.groupby(['maker'])]
        check = 1.0
        delta_ini = np.zeros(len(s0))
        while check > self.tol_fp:
            delta_new = delta_ini + (np.log(s0) - np.log(self.market_share(mid, delta_ini, xv, jf)))
            check = np.max(abs(delta_new - delta_ini))
            delta_ini = delta_new
        
        '''
        Calculate markups in a market at current parameter values (delta, sigmas)
        '''
        mrkup = self.market_share(mid, delta_new, xv, jf, markup=True)
        return {'delta': delta_new, 'markup': mrkup} 
    
    @staticmethod
    def get_markups(udic):
        dmat = udic['dmat']
        shares = udic['shares']
        alpha = udic['alpha']
        cut = 0
        markups = []
        for d in dmat:
            svec = shares[cut:cut+d.shape[0]] # shares of products from a firm
            d = np.linalg.inv(alpha* d)
            markups.append(np.matmul(d, svec))
            cut = cut + d.shape[0]
        return np.concatenate(tuple(markups), axis=0) # a J by 1 vector of product markups 
    
    def mean_utility(self,sigmas):
        """
        sigmas: an 1_D array with the shape (len(self.attributes_random), ), which contains
        the standard errors of random coefficients
        """
        df = self.df.copy()
        v_list = ['share', 'maker'] + self.attributes_random
        
        '''
        # step 1: solve mean utility (delta_j) from the fixed-point iteration
        '''
        df_list = [{'mid': int(mid), 'df': d[v_list], 'sigmas': sigmas} for mid, d in df.groupby(['trend'])]
        out = list(map(self.fixed_point, df_list))
                    
        '''
        step 2: uncover mean part of coefficients (beta_bar) from delta_j, which is equivalent to 
        running an IV estimation using delta_j as the dependent variable
        '''
        delta_j = tuple([i['delta'] for i in out])
        delta_j = np.concatenate(delta_j, axis=0) # an array with the shape(2217,)
        beta_bar = np.matmul(self.project_mat, delta_j) # uncover mean coefficients in demand model
        
        '''
        step 3: uncover ommited product attributes (xi_j) from delta_j and beta_bar
        '''
        xi_j = delta_j - np.matmul(self.Xmat, beta_bar)
        
        '''
        step 4: uncover cost-side parameters and cost shocks
        '''
        alpha = beta_bar[-1] # the last coefficient in beta_bar is the price coefficient
        inputs = [{'alpha': alpha, 'dmat': i['markup']['dmat'], 'shares': i['markup']['shares']} for i in out]
        out = tuple(map(self.get_markups, inputs))
        markups = np.concatenate(out, axis=0) # an array with the shape(2217,)
        y_s = self.df['price'] + markups
        gamma = np.matmul(self.PWMAT, y_s) # parameters of marginal cost equation
        
        """
        step 5: uncover omitted product attributes affecting marginal cost
        """
        omega = y_s - np.matmul(self.Mcmat, gamma)
        return {'beta_bar': beta_bar, 'xi_j': xi_j, 'gamma': gamma, 'omega': omega} 
    
    def GMM_obj(self, sigmas):
        
        res = self.mean_utility(sigmas)
        '''
        step 1: Demand-side moments: interact xi_j with instruments,which include exogenous regressors (veihicles' own
        exogenous attributes) and instruments for price (sum of attributes of competing products)
        '''
        xi_j = res['xi_j']
        moments_D = np.matmul(np.transpose(self.Zmat_D), xi_j) # an array with the shape (m, ), where m is the number of IVs
        
        """
        step 2: Supply-side moments: interact omega with supply-side moments which include exogeneous marginal cost shifters
        and their interactions
        """
        omega = res['omega']
        moments_S = np.matmul(np.transpose(self.Zmat_S), omega)
        
        '''
        step 5: compute the GMM objective function under the assumption that errors in demand and supply equations are independent
        '''
        f_D = np.matmul(moments_D, self.weight_mat_D)
        f_D = np.matmul(f_D, moments_D)
        f_S = np.matmul(moments_S, self.weight_mat_S)
        f_S = np.matmul(f_S, moments_S)
        
        return (1/len(self.df))* (f_D + f_S)
    
    def optimization(self, objfun, para):
        '''
        Parameters
        ----------
        objfun : a user defined objective function of para
            
        para : a 1-D array with the shape (k,), where k is the number of parameters.
        Returns
        -------
        dict
            A dictionary containing estimation results
        '''
        v = opt.minimize(objfun, x0=para, jac=None, method='BFGS', 
                          options={'maxiter': 1000, 'disp': True})  
        return {'obj':v.fun, "Coefficients": v.x}

if __name__ == "__main__":
    
    blp = BLP("/kaggle/input/blp-data/", "BLP_data.mat", "/kaggle/input/blp-str-data/", "BLP_data_str.mat")
    pini = np.ones(len(blp.attributes_random)) * 0
    x = blp.GMM_obj(pini)
    beta_ols = blp.ols()
    beta_iv = blp.iv()
    beta_hedonic= blp.hedonic_price()
    print(beta_ols.summary())
    print(beta_iv.summary())
    print(sm.OLS(blp.df['price'], blp.Zmat_D).fit().summary()) # first-stage regression in IV estimation
    print(beta_hedonic.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     279.2
Date:                Wed, 01 Mar 2023   Prob (F-statistic):          6.52e-232
Time:                        22:23:02   Log-Likelihood:                -3319.4
No. Observations:                2217   AIC:                             6651.
Df Residuals:                    2211   BIC:                             6685.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.0716      0.253    -39.822      0.0

In [14]:
v = blp.mean_utility(pini)
v['beta_bar']
v['gamma']


array([16.8001972 , -7.96731519, 10.37192072, -3.65111685,  8.38079655,
        0.15292663])

### Notes on the supply side in BLP estimation
In BLP paper, there are $F$ firms in a market (year) and each firm offers $J_f$ products. The toatl number of products in a market is therefore $J = \sum_f J_f$. For convenience in interpretation, we assume that the $J$ products are grouped and ordered by firm. The supply side in BLP paper is the following price equation of a market in matrix form

$$
P = MC + Markup = W \gamma - \Delta(P,X;\theta)^{-1} \times S(P,X;\theta) + \omega
$$

where \
$P$: a $J$ by 1 price vector; \
$X$: a $J$ by $K$ matrix of $K$ vehicle attributes excluding price; \
$W$: a $J$ by $L$ matrix of $L$ marginal cost shifters; \
$\theta$: demand side parameters; \
$\gamma$: cost side parameters;\
$S(P,X;\theta)$: a $J$ by 1 vector of simulated market shares given current parameter values; \
$\omega$: a $J$ by 1 vector of supply-side shocks; 
$\Delta(P,X;\theta)$: a $J$ by $J$ Jacobian matrix defined as 
\begin{equation}
\Delta(P,X;\theta)=diag(\Delta_1(P,X;\theta), \Delta_2(P,X;\theta), \cdots, \Delta_F(P,X;\theta)),
\end{equation}
in which $diag()$ denotes the operator of a diagonal matrix and a block in the blocked diagonal matrix $\Delta_f(P,X;\theta)$ takes the fowllowing form: 
<br>

$$
\Delta_f(P,X;\theta) = 
\begin{bmatrix}
 \frac{\partial s_1(P,X;\theta)}{\partial p_1} & \frac{\partial s_2(P,X;\theta)}{\partial p_1} & \cdots & \frac{\partial s_J(P,X;\theta)}{\partial p_1}\\
\frac{\partial s_1(P,X;\theta)}{\partial p_2} & \frac{\partial s_2(P,X;\theta)}{\partial p_2} & \cdots & \frac{\partial s_J(P,X;\theta)}{\partial p_2} \\
\vdots  & \vdots  & \ddots & \vdots \\
\frac{\partial s_1(P,X;\theta)}{\partial p_{J_f}} & \frac{\partial s_2(P,X;\theta)}{\partial p_{J_f}} & \cdots & \frac{\partial s_J(P,X;\theta)}{\partial p_{J_f}}
\end{bmatrix}
 $$
 <br>
 We can derive analytically the partial derivatives in the above Jacobian matrix. Remember that
\begin{equation}
s_j(P,X;\theta)= \int_{V_i}\frac{exp(X_j \bar{B} + \alpha p_j + X_jV_i)}{1+\sum_{j'}exp(X_{j'} \bar{B} + \alpha p_{j'} + X_{j'}V_i)} f(V_i;\Sigma)dV_i =  \int_{V_i}s_j(P,X;\theta|V_i)f(V_i;\Sigma)dV_i
\end{equation}

where $f(V_i;\Sigma)$ is a joint normal density function with zero means and a diaganol variance-covariance matrix $\Sigma = diag(\sigma_1,\sigma_2,\cdots,\sigma_K)$. Using Monte-Carlo integration, we have

\begin{equation}
\frac{\partial s_j(P,X;\theta)}{\partial p_{k}}=\int_{V_i}\frac{\partial s_j(P,X;\theta|V_i)}{\partial p_{k}}
f(V_i;\Sigma)dV_i \approx
    \begin{cases}
        \alpha R^{-1} \sum_{r=1}^{R} s_j(P,X;\theta|V_i^r) \times (1 - s_j(P,X;\theta|V_i^r)) & \text{if} j=k \\
        -\alpha R^{-1} \sum_{r=1}^{R} s_j(P,X;\theta|V_i^r)\times s_k(P,X;\theta|V_i^r) & \text{if} j \neq k        
    \end{cases}
\end{equation}


In [13]:
g = sm.OLS(blp.df['price'], blp.Mcmat).fit()
g.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     518.9
Date:                Wed, 01 Mar 2023   Prob (F-statistic):               0.00
Time:                        22:29:28   Log-Likelihood:                -7066.4
No. Observations:                2217   AIC:                         1.414e+04
Df Residuals:                    2211   BIC:                         1.418e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.1816      0.856     25.906      0.000      20.503      23.861
x1            -7.9559      0.911     -8.736      0.000      -9.742      -6.170
x2            10.3484      0.352     29.417      0.000       9.659      11.038
x3            -3.5793      1.190     -3.007      0.003      -5.913      -1.245
x4             8.3805      0.658     12.735      0.000       7.090       9.671
x5             0.1528      0.028      5.412      0.000       0.097       0.208
==============================================================================
Omnibus:                     1156.789   Durbin-Watson:                   0.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9328.649
Skew:                           2.341   Prob(JB):                         0.00
Kurtosis:                      11.891   Cond. No.                         150.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""